## Log a Keras Model

Log a Keras Model with Katonic SDK's Log package.

In [1]:
!pip install tensorflow -q

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


### Imports

In [2]:
import os

import tensorflow as tf
from katonic.log.logmodel import LogModel

2022-06-10 11:04:29.995534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 11:04:29.995578: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Experiment Name

In [3]:
experiment_name= "keras_model_logging"

### Initiate LogModel with experiment name

In [4]:
lm = LogModel(experiment_name)

2022/06/10 11:04:36 INFO mlflow.tracking.fluent: Experiment with name 'keras_model_logging' does not exist. Creating a new experiment.


### Metadata of the created / existing experiment

In [5]:
# experiment id
exp_id = lm.id

print("experiment name: ", lm.name)
print("experiment location: ", lm.location)
print("experiment id: ", lm.id)
print("experiment status: ", lm.stage)

experiment name:  keras_model_logging
experiment location:  s3://models/21
experiment id:  21
experiment status:  active


### Artifact path where you want to log your model|

In [6]:
artifact_path = "mnist-keras-model"

### Define your Keras Model

In [7]:
mnist = tf.keras.datasets.mnist

In [8]:
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

In [9]:
X_valid, X_train = X_train_full[:5000] / 255, X_train_full[5000:] / 255
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255

In [10]:
LAYERS = [
    tf.keras.layers.Flatten(input_shape=(28, 28), name='inputLayer'),
    tf.keras.layers.Dense(300, activation='relu', name='hiddenLayer1'),
    tf.keras.layers.Dense(100, activation='relu', name='hiddenLayer2'),
    tf.keras.layers.Dense(10, activation='softmax', name='outputLayer')
]

In [11]:
model_clf = tf.keras.models.Sequential(LAYERS)

2022-06-10 11:04:40.814706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-10 11:04:40.814746: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-10 11:04:40.814770: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ws-697d0210-73d1-4e46-9643-136c4d0acc29-0): /proc/driver/nvidia/version does not exist
2022-06-10 11:04:40.815062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
loss_function = 'sparse_categorical_crossentropy'
optimizers = 'SGD'
metric = ['accuracy']

model_clf.compile(
    loss=loss_function, 
    optimizer=optimizers, 
    metrics=metric
)

In [13]:
ckpt_path = "data/model_ckpt.h5"

In [14]:
ckpt_cb = tf.keras.callbacks.ModelCheckpoint(ckpt_path, save_best_only=True)

### Train your Keras Model

In [15]:
EPOCHS = 5
VALIDATION = (X_valid, y_valid)

history = model_clf.fit(
    X_train, y_train, 
    epochs=EPOCHS, 
    validation_data=VALIDATION,
    callbacks=[ckpt_cb]
)

2022-06-10 11:04:43.299668: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 172480000 exceeds 10% of free system memory.


Epoch 1/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6052 - accuracy: 0.8451 - val_loss: 0.3146 - val_accuracy: 0.9132
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2917 - accuracy: 0.9167 - val_loss: 0.2421 - val_accuracy: 0.9310
Epoch 3/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2393 - accuracy: 0.9310 - val_loss: 0.2072 - val_accuracy: 0.9412
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2049 - accuracy: 0.9415 - val_loss: 0.1823 - val_accuracy: 0.9510
Epoch 5/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1797 - accuracy: 0.9484 - val_loss: 0.1611 - val_accuracy: 0.9552


### Evalute Model

In [16]:
res = model_clf.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - loss: 0.1652 - accuracy: 0.9527 - 618ms/epoch - 2ms/step


In [17]:
model_clf.metrics_names

['loss', 'accuracy']

In [18]:
model_mertics = {
    model_clf.metrics_names[0]: res[0],
    model_clf.metrics_names[1]: res[1]
}

In [19]:
model_clf.predict(X_test)

313/313 [==============================] - 1s 2ms/step


array([[6.27060872e-05, 9.70111046e-07, 3.12100863e-04, ...,
        9.98590946e-01, 3.82058788e-05, 1.28434302e-04],
       [7.13106536e-04, 2.15316424e-04, 9.87813830e-01, ...,
        6.23296215e-09, 3.96948977e-04, 4.49186928e-08],
       [4.22985140e-05, 9.79464889e-01, 6.88691530e-03, ...,
        3.29923746e-03, 3.15089524e-03, 5.46108349e-04],
       ...,
       [1.01178415e-07, 9.96495046e-08, 2.85778765e-06, ...,
        6.52536255e-05, 1.09955529e-03, 3.01748863e-03],
       [8.96370111e-05, 1.14575123e-04, 3.09662164e-06, ...,
        6.61108231e-07, 9.51778144e-03, 8.68939514e-06],
       [1.39258655e-05, 5.28123989e-09, 8.24687231e-05, ...,
        2.97761982e-10, 7.52132564e-07, 3.19122826e-08]], dtype=float32)

### Log Custom Model

In [21]:
lm.model_logging(
    model_name="mnistkeras",
    model_type="keras",
    model=model_clf,
    artifact_path=artifact_path,
    current_working_dir=f'{os.getcwd()}/keras_model_logging.ipynb',
    metrics=model_mertics
)

INFO:tensorflow:Assets written to: /tmp/tmplt7seh2_/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmplt7seh2_/model/data/model/assets


Model artifact logged to: s3://models/21/29f0add02a674b6eb9c5da21e495729e/artifacts/keras_model_logging_21_mnist-keras-model_mnistkeras


In [22]:
df_runs = lm.search_runs(exp_id)
print("Number of runs done : ", len(df_runs))

Number of runs done :  1


In [23]:
df_runs.head()

,artifact_uri,end_time,experiment_id,metrics.accuracy,metrics.loss,run_id,run_name,start_time,status,tags.mlflow.log-model.history
0,s3://models/21/29f0add02a674b6eb9c5da21e495729...,2022-06-10 11:05:42.707000+00:00,21,0.9527,0.165211,29f0add02a674b6eb9c5da21e495729e,keras-model,2022-06-10 11:05:35.834000+00:00,FINISHED,"[{""run_id"": ""29f0add02a674b6eb9c5da21e495729e""..."


### Load your logged keras model with URI

In [28]:
df_runs['artifact_uri'][0]

's3://models/21/29f0add02a674b6eb9c5da21e495729e/artifacts'

In [29]:
model_uri='s3://models/21/29f0add02a674b6eb9c5da21e495729e/artifacts/keras_model_logging_21_mnist-keras-model_mnistkeras'

In [30]:
loaded_model = lm.model_loading(model_uri)

### Prediction with Loaded model

In [31]:
loaded_model.predict(X_test)

313/313 [==============================] - 1s 2ms/step


array([[6.27060872e-05, 9.70111046e-07, 3.12100863e-04, ...,
        9.98590946e-01, 3.82058788e-05, 1.28434302e-04],
       [7.13106536e-04, 2.15316424e-04, 9.87813830e-01, ...,
        6.23296215e-09, 3.96948977e-04, 4.49186928e-08],
       [4.22985140e-05, 9.79464889e-01, 6.88691530e-03, ...,
        3.29923746e-03, 3.15089524e-03, 5.46108349e-04],
       ...,
       [1.01178415e-07, 9.96495046e-08, 2.85778765e-06, ...,
        6.52536255e-05, 1.09955529e-03, 3.01748863e-03],
       [8.96370111e-05, 1.14575123e-04, 3.09662164e-06, ...,
        6.61108231e-07, 9.51778144e-03, 8.68939514e-06],
       [1.39258655e-05, 5.28123989e-09, 8.24687231e-05, ...,
        2.97761982e-10, 7.52132564e-07, 3.19122826e-08]], dtype=float32)